# Preprocessing: PART II (_MAIN)

**INPUT: _filt.fif files**

1. Fixes the triggers for the _MAIN task
    - Restores missing target onset (20) and go signal (30) triggers from PHOTODIODE CHANNEL
    - Reassign trigger numbers and give them names that make sense 
    - Adds missing start and end triggers for trials

    (see the script for more detailed steps)

2. Marks incomplete trials as bad

**OUTPUT: _filt_clean.fif file**

**Proceed preprocessing using 'fixing_triggers_BL.ipynb' file!**

In [1]:
import mne
import mne.io.fiff
import os
from utils import check_paths
import re
import copy
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
%matplotlib qt

**GROUP ANALYSIS**

ANALYSIS STEPS WITH FUNCTIONS

PHOTODIODE CHANNEL:
Extract triggres for target onset (20) and go signal (30)

In [2]:
# This function converts photodiode responses to events
# INPUT: raw_filt.fif file, photodiode channel name, event id
# OUTPUT: np.array with events

def photo_to_events(raw_filt: mne.io.fiff.raw.Raw, photo_chan: str, event_id: int) -> np.array:
    
    ############# CREATE TRIGGERS WHEN PHOTODIODE EXCEEDS CROSSES THE THRESHOLD #############
    # Get the photodiode channel
    photodiode_channel = raw_filt.copy().pick_channels([photo_chan])

    # Get the data from the photodiode channel
    photodiode_data = photodiode_channel.get_data()[0]  # Get the first (and only) channel data

    # Set a threshold to detect event triggers (you may need to adjust this threshold)
    threshold = np.mean(photodiode_data) + 4 * np.std(photodiode_data)

    # Find indices where the signal crosses the threshold (rising edges)
    photodiode_events = np.where(np.diff(photodiode_data > threshold, prepend=0))[0]

    # Create the events array (event_id can be customized)
    photo_events = np.c_[photodiode_events, np.zeros(len(photodiode_events)), np.full(len(photodiode_events), event_id)].astype(int)
    ####################################################


    ############# FIND MIN TIME DIFFERENCE WHEN PHOTODIODE CROSSES THE THRESHOLD #############
    # Determine min time diff btw start and end of photodiode peak
    time_dists = []
    for i in range(len(photo_events)-1):
        if i%2 == 0:
            time_dist = photo_events[i+1, 0] - photo_events[i, 0]
            time_dists.append(time_dist)

    min_time_diff = min(time_dists)
    ####################################################


    ############# KEEP ONLY THE FIRST PHOTODIODE EVENT IN EACH PAIR #############
    # Set a threshold that determines proximity (based on sampling rate and expected event duration)
    # Initialize an empty list to store the filtered events
    photo_events_filt = []
    step_from_min = 5 # Check in with the data, if that's an appropriate step to use

    # Loop through the events and keep only the start event of each pair
    for i in range(len(photo_events) - 1):
        if i%2 == 0:
            # Compare the current event to the next one
            if photo_events[i + 1, 0] - photo_events[i, 0] > (min_time_diff - step_from_min):
                photo_events_filt.append(photo_events[i])

    # Convert the list back to a NumPy array
    photo_events_filt = np.array(photo_events_filt)
    ####################################################


    return photo_events_filt
    

Merge events from STIM with events from ANNOTATIONS

In [3]:
# This function merger events from PHOTODIODE with events from ANNOTATIONS
# INPUT: raw_filt.fif file, photodiode events
# OUTPUT: tuple with events array and events dictionary

def merge_photo_events(raw_filt: mne.io.fiff.raw.Raw, photo_events: np.array) -> tuple:
    
    ####### Merge two arrays of events #######
    # Read events stored in the raw file
    events_annot = mne.events_from_annotations(raw_filt)

    # Extract the array and dictionary from events_annot
    events_annot_array = events_annot[0]
    events_annot_dict = events_annot[1]

    # Merge the two event arrays using np.vstack and sort the merged events array by the first column (time)
    merged_events = np.vstack((events_annot_array, photo_events))
    merged_events = merged_events[merged_events[:, 0].argsort()]

    # Add '99' to the event dictionary with a label
    events_annot_dict['99'] = 99
    ####################################################


    ####### If there is a trigger near the Photodiode trigger, remove protodiode trigger #######
    # Determine mean time diff btw photodiode peak and preceeding eeg trigger (excluding too long time distances)
    time_dists_photo = []
    long_dist = 50 #! arbitrary - check if it makes sense based on the data 

    for i in range(len(merged_events) - 1):
        if merged_events[i+1, 2] == 99:
            time_dist = merged_events[i+1, 0] - merged_events[i, 0]
            if time_dist < long_dist:
                time_dists_photo.append(time_dist)

    mean_time_diff_photo = np.mean(time_dists_photo)

    # Set a threshold that determines proximity (based on your sampling rate and expected event duration)
    rows_to_filt = []
    step_from_mean = 5 # Check in with the data, if that's an appropriate step to use

    # Loop through the events and keep only the start event of each pair
    for i in range(len(merged_events) - 1):
                if merged_events[i+1, 2] == 99:
                    # Compare the current event to the next one
                    if merged_events[i+1, 0] - merged_events[i, 0] <=  (mean_time_diff_photo + step_from_mean):
                        rows_to_filt.append(i+1)
                    elif merged_events[i, 2] in range(1,8):
                         rows_to_filt.append(i+1)
    # get rid of two similar arrays if there are any (sometimes the case with the impedance check trigger 0) 
    for i in range(len(merged_events) - 1):
        #  if merged_events[i+1][0] == merged_events[i][0]:
        #       rows_to_filt.append(i+1)
        if merged_events[i][2] == merged_events[i+1][2]:
            # print(merged_events[i][2], merged_events[i+1][2])
            # print('two SAME triggers in a row!')
            dist = merged_events[i+1][0] - merged_events[i][0]
            # print(dist)
            if dist < 5:
                print(merged_events[i][2], '- dist too short!')
                rows_to_filt.append(i+1)
         
    # Delete the rows with duplicated events
    merged_events_filt = np.delete(merged_events, rows_to_filt, axis=0)
    ####################################################

    # Create a new tuple with the new dictionary
    events = (merged_events_filt, events_annot_dict)

    return events


Reassign values to the triggers

In [4]:
# This function assing names to the values (triggers) in EVENTS DICT corresponding to keys and replaces old trigger names with new ones in EVENTS ARRAY 
# INPUT: events tuple
# OUTPUT: tuple with renamed events
def rename_events_after_triggers(events: tuple) -> tuple:
    
    new_events = copy.deepcopy(events)
    
    # Assing events the values corresponding to the keys names
    for key, value in new_events[1].items():
        new_value = int(re.search(r'\d+', key).group())
        new_events[1][key] = new_value

    # Create a mapping from the old event values to the new event values
    value_mapping = {events[1][key]: new_events[1][key] for key in events[1] if key in new_events[1]}

    # Replace the values in the third column of the events array
    for old_value, new_value in value_mapping.items():
        new_events[0][:, 2][new_events[0][:, 2] == old_value] = new_value

    return new_events

MAIN TASK - give keys in the dictionary meaningful names

1:	start of practice
2:	start of baseline
3:	start of first rotated block
4:	start of second rotated block
5:	start of post block
6:	start of first deadapt block
7:	start of second deadapt block

8:	trial start
9:	trial end

20:	target onset
30:	go signal onset
40:	reach end

111:	bad trial - moved in response to display of target
112:	bad trial - too slow 	%!!! CHANGED from 222
113:	bad trial - didnt react within 1s 	%!!! CHANGED from 333

In [5]:
# This function assings meaningful names to the keys in EVENTS DICT
# INPUT: events tuple
# OUTPUT: tuple with new EVENTS DICT keys

def make_sense(events: tuple, names_dict: dict) -> tuple:
    
    # Create an inverse mapping of main_dict
    inverse_dict = {v: k for k, v in names_dict.items()}

    # Replace keys in new_events with corresponding keys from main_dict
    replaced_keys_dict = {}
    for old_key, value in events[1].items():
        # Look up the corresponding new key using the value
        new_key = inverse_dict.get(value, old_key)  # Default to old_key if value not found in main_dict
        replaced_keys_dict[new_key] = value

    # Create a new tuple with the new dictionary
    new_events = (events[0], replaced_keys_dict)
    
    return new_events


Add missing end of trial events

In [6]:
# This function adds end of trial events (9) to BAD trials (111 & 77)
# INPUT: events tuple
# OUTPUT: updated events array

def add_end_to_bad(events: tuple, bad_triggers=[None]) -> np.array:
    
    # SANITY CHECK: min_time should not exceed 10 (see the following cell)
    time_dists = []

    for i in range(len(events[0][:-1, 0])):
        time_dist = events[0][i+1, 0] - events[0][i, 0]
        time_dists.append(time_dist)

    min_time = min(time_dists)
    # print(time_dists)
    print('Min time:', min_time)

    if min_time < 10:
        raise ValueError('Min_time is less than 10 - adjust the timing for new events!')

    add_events = []

    # Loop through events, find 111 triggers, and add trigger 9
    for event in events[0]:
        if event[2] in bad_triggers.values():  # 111, 112, 113 trigger code
            # Add min distance between 111 and new 9
            add_event_time = event[0] + 10

            # Create a new event with the trigger code 9
            add_event = [add_event_time, 0, events[1]['trial_end']]  # [time, 0, trigger code 9]

            # Append the new event to the list of new events
            add_events.append(add_event)

    # Convert add_events list to a NumPy array
    add_events = np.array(add_events, dtype=int)

    # Combine the original events with the new events
    events_array = np.vstack((events[0], add_events))

    # Sort the modified events by the time column (first column)
    events_array = events_array[np.argsort(events_array[:, 0])]

    return events_array

Introduce start (8) and end (9) of trial triggers if there are none

In [7]:
# This function adds start (8) and end (9) of trial events, if they are missing
# INPUT: events array
# OUTPUT: updated events array

def add_start_and_end(events_array: np.array, events_ids: dict) -> np.array:

    #!!! modified_events -> new_events_array !!!

    ####### Check mean time distance between start (8) and end (9) of trial events #######
    # SANITY CHECK: if mean significantly deviates from 100 - check the data
    to_avg = []
    for i in range(len(events_array) - 1):
        if events_array[i, 2] == 9:
            if events_array[i+1, 2] == 8:
                if events_array[i+1, 0] - events_array[i, 0] < 1000:
                    to_avg.append(events_array[i+1, 0] - events_array[i, 0])

    print('Mean:', np.mean(to_avg))

    if int(np.mean(to_avg)) not in range(90,110):
        raise ValueError('Mean deviates from 100 - check the data!')
    ###########################################################################


    ####### Add missing start (8) and end (9) of trial events #######
    upd_events = []
    for i in range(len(events_array) - 1):

        # if there's no start
        if events_array[i, 2] == 9:
            if events_array[i+1, 2] not in range(9):
                # print(i,'- no start')
                
                # Add mean timing btw end (9) and start (8)
                new_event_time = events_array[i, 0] + int(np.mean(to_avg))

                # Create a new event with the trigger code 8
                new_event = [new_event_time, 0, events_ids['trial_start']]  # [time, 0, trigger code 8]

                # Append the new event to the events_array array
                upd_events.append(new_event)
                # print(new_event)

        # if there's no start after break
        elif events_array[i, 2] in range(1,8):
            if events_array[i+1, 2] != 8:
                # print(i+1,'- no start after break')
                
                # Add mean timing btw end (9) and start (8)
                new_event_time = events_array[i, 0] + int(np.mean(to_avg))

                # Create a new event with the trigger code 8
                new_event = [new_event_time, 0, events_ids['trial_start']]  # [time, 0, trigger code 9]

                # Append the new event to the events_array array
                upd_events.append(new_event)
                # print(new_event)
        
        # if there's no end
        elif events_array[i+1, 2] == 8:
            if events_array[i, 2] not in range(1,10):
                # print(i,'- no end')
                
                # Subtract mean timing btw end (9) and start (8)
                new_event_time = events_array[i+1, 0] - int(np.mean(to_avg))

                # Create a new event with the trigger code 9
                new_event = [new_event_time, 0, events_ids['trial_end']]  # [time, 0, trigger code 9]

                # Append the new event to the events_array array
                upd_events.append(new_event)
                # print(new_event)

        # if there's no end before break
        elif events_array[i+1, 2] in range(2,8):
            if events_array[i, 2] != 9:
                # print(i+1,'- no end before break')
                
                # Subtract mean timing btw end (9) and start (8)
                new_event_time = events_array[i, 0] - int(np.mean(to_avg))

                # Create a new event with the trigger code 9
                new_event = [new_event_time, 0, events_ids['trial_end']]  # [time, 0, trigger code 9]

                # Append the new event to the events_array array
                upd_events.append(new_event)
                # print(new_event)
        
    # if no end for last trial
    if events_array[-1, 2] == 0 and events_array[-2, 2] != 9:
            
            # Subtract mean timing btw end (9) and start (8)
            new_event_time = events_array[-1, 0] - 5

            # Create a new event with the trigger code 9
            new_event = [new_event_time, 0, events_ids['trial_end']]  # [time, 0, trigger code 9]

            # Append the new event to the events_array array
            upd_events.append(new_event)
            # print(new_event)

    # Convert new_events list to a NumPy array
    # upd_events = np.array(upd_events, dtype=int)
    upd_events = np.array(upd_events, dtype=int) if upd_events else np.empty((0, 3), dtype=int)

    # Combine the original events with the new events
    events_array = np.vstack((events_array, upd_events))

    # Sort the modified events by the time column (first column)
    events_array = events_array[np.argsort(events_array[:, 0])]
                
    return events_array

Rename the 99 triggers to 20 or 30

In [8]:
# This function converts photodiode events (99) to either target_on (20) or go signal (30)
# INPUT: events array
# OUTPUT: None or ValueError if events counts don't match

def get_target_and_go_from_photo(events_array: np.array, events_ids: dict) -> tuple:

    # Initialize an empty list to store the filtered events
    rows_to_filt = []

    # replace 99 with 20 and 30 and filter 99 triggerch that do not match the pattern
    for i in range(len(events_array) - 1):

        if events_array[i, 2] == 99:
            if events_array[i-1, 2] == 8:
                if events_array[i+1, 2] == 9:
                    rows_to_filt.append(i)
                elif events_array[i+1, 2] == 30 or events_array[i+1, 2] == 99:
                    events_array[i, 2] = 20
            elif events_array[i-1, 2] == 20 and events_array[i+1, 2] == 40:
                events_array[i, 2] = 30
            else:
                rows_to_filt.append(i)
    print(rows_to_filt)
    print('LENGTH:', len(rows_to_filt))

    for i in range(1, len(events_array) - 1):
        if events_array[i, 2] == 99:
            if events_array[i-1, 2] == 8 and events_array[i+1, 2] == 9:
                rows_to_filt.append(i-1)
                rows_to_filt.append(i+1)
    print(rows_to_filt)
    print('LENGTH:', len(rows_to_filt))

    # Sort and remove duplicates
    rows_to_filt = sorted(set(rows_to_filt))
    print(rows_to_filt)

    # delete 99 rows from events array and 'photodiode' value from events dict
    events_array_filt = np.delete(events_array, rows_to_filt, axis=0)
    events_ids.pop("photodiode")

    # save final version of events tuple
    events_final_to_save = (events_array_filt, events_ids)
    
    return events_final_to_save

CHECK EVENTS COUNTS

In [9]:
# This function counts number of events and checks if they match
# INPUT: events array
# OUTPUT: None or ValueError if events counts don't match

def check_events_counts(events_array: np.array) -> None:

    # Convert the column with trigger names to a pandas Series and get value counts
    value_counts = pd.Series(events_array[:,2]).value_counts()

    # Get counts for specific numbers (if they exist, otherwise return 0)
    count_8 = value_counts.get(8, 0) # start of trial
    count_9 = value_counts.get(9, 0) # end of trial
    count_111 = value_counts.get(111, 0) # bad early
    count_77 = value_counts.get(77, 0) # bad late

    # Check if num of 8 equals num of 9
    if count_8 != count_9:
        raise ValueError(f"Count of 8 = {count_8} does not equal count of 9 = {count_9} !")

    print(f'Total number of trials: {count_8}')

    # Check if num of 111 + num of 77 equals num of 8
    if count_8 - (count_111 + count_77) != 520:
        raise Warning(f"Count of 8 - (111 + 77) = {count_8 - (count_111 + count_77)} does not equal 520 !")

    print("ALL GOOD: Events counts match!")


Marking the trials that do not contain [8 20 30 40 9] pattern as BAD

In [10]:
# This function annotates trials that do not match the pattern [8, 20, 30, 40, 9]
# INPUT: raw_filt fif file, events array
# OUTPUT: annotated raw_filt fif file

def annotate_bad_trials(raw_filt: mne.io.fiff.raw.Raw, events_array: np.array) -> mne.io.fiff.raw.Raw:

    # The correct trigger sequence for a normal trial
    correct_sequence = [8, 20, 30, 40, 9]

    # Get the sampling frequency from the raw EEG data
    sfreq = raw_filt.info['sfreq']

    # Find indices of all trial starts (trigger 8)
    trial_start_indices = np.where(events_array[:, 2] == 8)[0]

    # List to store annotations for bad segments
    bad_segs = []

    # Iterate through each trial start index
    for start_idx in trial_start_indices:
        # Find the index of the corresponding '9' trigger (trial_end)
        trial_end_idx = np.where(events_array[start_idx:, 2] == 9)[0]
        
        if len(trial_end_idx) > 0:  # Make sure '9' trigger exists after '8'
            trial_end_idx = trial_end_idx[0] + start_idx  # Adjust relative to start_idx
            
            # Extract the potential trigger sequence starting from this trial start
            trigger_sequence = events_array[start_idx:trial_end_idx+1, 2]  # Include '9' in the sequence
            
            # Check if the extracted sequence matches the correct sequence
            if not np.array_equal(trigger_sequence, correct_sequence):
                # Mark this segment as bad from trigger 8 to trigger 9
                start_time = events_array[start_idx, 0] / sfreq  # Convert sample point to time (in seconds)
                end_time = events_array[trial_end_idx, 0] / sfreq  # Time of trigger 9
                
                # Append the bad segment as an annotation
                bad_segs.append((start_time, end_time - start_time, 'bad'))

    # Add an annotation for everything before the first trigger '2' (baseline)
    baseline_idx = np.where(events_array[:, 2] == 2)[0]  # Find the index of trigger 2
    if len(baseline_idx) > 0:
        first_baseline_idx = baseline_idx[0]
        first_baseline_time = events_array[first_baseline_idx, 0] / sfreq  # Time of the first trigger 2
        
        # Mark from the start of the recording (time 0) to the time of the first trigger 2
        bad_segs.append((0, first_baseline_time, 'bad'))
    else:
        ValueError(f'{sub_name}: No trigger for Baseline block!')

    # Convert bad_segs list to MNE's Annotations format
    onsets, durations, descriptions = zip(*bad_segs)
    bad_annotations = mne.Annotations(onset=onsets, duration=durations, description=descriptions)

    # Add the new annotations to the raw data
    raw_filt.set_annotations(bad_annotations)

    return raw_filt

_________________________

ANALYSIS START

In [ ]:
subs = ['s1_pac_sub43', 's1_pac_sub58', 's1_pac_sub59', 's1_pac_sub61', 's1_pac_sub63',
        's1_pac_sub64', 's1_pac_sub67', 's1_pac_sub71', 's1_pac_sub72', 's1_pac_sub76',
        's1_pac_sub77']

In [11]:
# MAIN dictionary with events

main_dict = {'0, Impedance': 0,
            'bad_early': 111,
            'bad_late': 77,
            'prac': 1,
            'baseline': 2,
            'target_on': 20,
            'adapt1': 3,
            'go_on': 30,
            'adapt2': 4,
            'reach_end': 40,
            'postadapt': 5,
            'deadapt1': 6,
            'deadapt2': 7,
            'trial_start': 8,
            'trial_end': 9,
            'photodiode': 99}

bad_triggers = {key: value for key, value in main_dict.items() if 'bad_' in key}

In [ ]:
# Define the source directory and the destination directory
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
group = 'O' # Y or O
subs_dir = os.path.join(eeg_data_dir, group)
subs = os.listdir(subs_dir)
task = '_MAIN' # '_BL' or '_MAIN'

for sub_name in subs: # os.listdir(subs_dir) OR ['s1_pac_sub00']

    filt_dir = os.path.join(subs_dir, sub_name, 'preproc', 'filt')
    eeg_data_path = os.path.join(filt_dir, f'{sub_name}{task}_filt.fif')
    raw_filt = mne.io.read_raw_fif(eeg_data_path, preload=True)

    print(f'{sub_name}: PROCESSING TRIGGERS...')

    photo_events_filt = photo_to_events(raw_filt, photo_chan='BIP3', event_id=99) # This function converts photodiode responses to events
    events = merge_photo_events(raw_filt, photo_events_filt) # This function merger events from PHOTODIODE with events from ANNOTATIONS
    new_events = rename_events_after_triggers(events) # This function assing names to the values (triggers) in EVENTS DICT corresponding to keys
                                                      # and replaces old trigger names with new ones in EVENTS ARRAY 


    new_events = make_sense(new_events, main_dict) # This function assings meaningful names to the keys in EVENTS DICT

    new_events_array = add_end_to_bad(new_events, bad_triggers) # This function adds end of trial events (9) to BAD trials (111 & 77)
    upd_events_array = add_start_and_end(new_events_array, new_events[1]) # This function adds start (8) and end (9) of trial events, if they are missing
    events_final_to_save = get_target_and_go_from_photo(upd_events_array, new_events[1]) # This function converts photodiode events (99) to either target_on (20) or go signal (30)

    # Check if number of events match
    try:
        check_events_counts(events_final_to_save[0]) # This function counts number of events and checks if they match
    except ValueError as e:
        print(f"Error: {e}")
    except Warning as w:
        print(f"Warning: {w}")


    # Save events tuple as a pickle file
    with open(os.path.join(filt_dir, f'{sub_name}{task}_events.pkl'), 'wb') as pickle_file:
        pickle.dump(events_final_to_save, pickle_file)
    print(f"{sub_name}: Events SAVED successfully!")

    raw_filt = annotate_bad_trials(raw_filt, events_final_to_save[0]) # This function annotates trials that do not match the pattern [8, 20, 30, 40, 9]

    #! Check the raw data that contains annotations for bad segments
    raw_filt.plot(events=events_final_to_save[0])
    
    # save annotated file
    raw_filt.save(os.path.join(filt_dir, f'{sub_name}{task}_filt_events.fif'), overwrite=True)
    print(f'{sub_name}: Processing finished successfully - triggers recovered and bad trials marked!')

Opening raw data file D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub53\preproc\filt\s1_pac_sub53_MAIN_filt.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60)  idle
    Range : 0 ... 1883339 =      0.000 ...  3766.678 secs
Ready.
Reading 0 ... 1883339  =      0.000 ...  3766.678 secs...


C:\Users\a1902989\AppData\Local\Temp\ipykernel_28340\2196874148.py:11: RuntimeWarning: This filename (D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub53\preproc\filt\s1_pac_sub53_MAIN_filt.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_filt = mne.io.read_raw_fif(eeg_data_path, preload=True)


s1_pac_sub53: PROCESSING TRIGGERS...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: [np.str_('0, Impedance'), np.str_('1'), np.str_('111'), np.str_('2'), np.str_('20'), np.str_('3'), np.str_('30'), np.str_('4'), np.str_('40'), np.str_('5'), np.str_('6'), np.str_('7'), np.str_('77'), np.str_('8'), np.str_('9')]
Min time: 16
Mean: 100.26510721247563
[1, 1462]
LENGTH: 2
[1, 1462, 1461, 1463]
LENGTH: 4
[1, 1461, 1462, 1463]
Total number of trials: 569
ALL GOOD: Events counts match!
s1_pac_sub53: Events SAVED successfully!
Using matplotlib as 2D backend.
Writing D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub53\preproc\filt\s1_pac_sub53_MAIN_filt_events.fif


C:\Users\a1902989\AppData\Local\Temp\ipykernel_28340\2196874148.py:47: RuntimeWarning: This filename (D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub53\preproc\filt\s1_pac_sub53_MAIN_filt_events.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_filt.save(os.path.join(filt_dir, f'{sub_name}{task}_filt_events.fif'), overwrite=True)


Closing D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub53\preproc\filt\s1_pac_sub53_MAIN_filt_events.fif
[done]
s1_pac_sub53: Processing finished successfully - triggers recovered and bad trials marked!
Opening raw data file D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub54\preproc\filt\s1_pac_sub54_MAIN_filt.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60)  idle
    Range : 0 ... 1991855 =      0.000 ...  3983.710 secs
Ready.
Reading 0 ... 1991855  =      0.000 ...  3983.710 secs...


C:\Users\a1902989\AppData\Local\Temp\ipykernel_28340\2196874148.py:11: RuntimeWarning: This filename (D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub54\preproc\filt\s1_pac_sub54_MAIN_filt.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_filt = mne.io.read_raw_fif(eeg_data_path, preload=True)


s1_pac_sub54: PROCESSING TRIGGERS...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: [np.str_('0, Impedance'), np.str_('1'), np.str_('111'), np.str_('2'), np.str_('20'), np.str_('3'), np.str_('30'), np.str_('4'), np.str_('40'), np.str_('5'), np.str_('6'), np.str_('7'), np.str_('77'), np.str_('8'), np.str_('9')]
Min time: 50
Mean: 100.1598440545809
[1, 1011]
LENGTH: 2
[1, 1011, 1010, 1012]
LENGTH: 4
[1, 1010, 1011, 1012]
Total number of trials: 628
ALL GOOD: Events counts match!
s1_pac_sub54: Events SAVED successfully!
Writing D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub54\preproc\filt\s1_pac_sub54_MAIN_filt_events.fif


C:\Users\a1902989\AppData\Local\Temp\ipykernel_28340\2196874148.py:47: RuntimeWarning: This filename (D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub54\preproc\filt\s1_pac_sub54_MAIN_filt_events.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_filt.save(os.path.join(filt_dir, f'{sub_name}{task}_filt_events.fif'), overwrite=True)


Closing D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub54\preproc\filt\s1_pac_sub54_MAIN_filt_events.fif
[done]
s1_pac_sub54: Processing finished successfully - triggers recovered and bad trials marked!


In [15]:
new_events[1]

{'0, Impedance': 0,
 'prac': 1,
 'bad_early': 111,
 'baseline': 2,
 'target_on': 20,
 'adapt1': 3,
 'go_on': 30,
 'adapt2': 4,
 'reach_end': 40,
 'postadapt': 5,
 'deadapt1': 6,
 'deadapt2': 7,
 'bad_late': 77,
 'trial_start': 8,
 'trial_end': 9,
 'photodiode': 99}

____________

SANITY CHECKS

In [105]:
pd.Series(events_final_to_save[0][:, 2]).value_counts()

8      543
20     543
9      543
30     493
40     477
111     50
77      11
0        2
1        2
2        1
3        1
4        1
5        1
99       1
6        1
7        1
Name: count, dtype: int64

In [77]:
for i in events_final_to_save[0]:
    print(i)

[0 0 0]
[18142     0     1]
[18395     0     8]
[19613     0    20]
[19962     0   111]
[19972     0     9]
[21063     0     8]
[22438     0    20]
[23088     0    30]
[23712     0    40]
[23872     0     9]
[23973     0     8]
[28496     0    20]
[29429     0    30]
[29721     0    40]
[29880     0     9]
[29980     0     8]
[33646     0    20]
[33880     0    30]
[34071     0    40]
[34230     0     9]
[34331     0     8]
[41204     0    20]
[41521     0   111]
[41531     0     9]
[42621     0     8]
[46346     0    20]
[46896     0    30]
[47438     0    40]
[47597     0     9]
[47697     0     8]
[50104     0    20]
[50579     0   111]
[50589     0     9]
[51680     0     8]
[53321     0    20]
[53738     0    30]
[54171     0    40]
[54330     0     9]
[54430     0     8]
[55787     0    20]
[56538     0    30]
[57062     0    77]
[57072     0     9]
[58163     0     8]
[60896     0    20]
[61271     0    30]
[61630     0    40]
[61788     0     9]
[61888     0     8]
[63679     0

In [145]:
for i in range(len(events_final_to_save[0]) - 1):
    if events_final_to_save[0][i+1][2] == 9 and events_final_to_save[0][i][2] == 8:
        print(events_final_to_save[0][i], 'something is wrong...')

[1251784       0       8] something is wrong...
